# Terceira lista de visão computacional
# Lucas Heron Santos Anchieta
# Ruan Tenório de Melo

## Importações

In [1]:
import cv2 as cv
import numpy as np

## 1. Escolha uma das metodologias que você implementou na segunda lista para gerar correspondências entre um par de imagens. Aplique-a em 5 pares de imagens (com sobreposição) para calcular suas homografias, e aplique-as para gerar panoramas entre os pares de imagens (um panorama por par).
## Obs.: nessa questão, não é permitido usar a API de alto nível Stitcher.
## Dica: use a função warpPerspective da OpenCV.

## 2. Repita a questão anterior com 5 trios de imagens (com sobreposição 2 a 2), alinhando as imagens no plano da primeira imagem. Repita o mesmo alinhando no plano da segunda imagem, e da terceira imagem. Note que aqui será necessário compor as transformações de homografia em alguns casos, ou calcular inversas.

## 3. Considere a imagem soccer.jpg (./img/soccer.jpg) em anexo no Google Classroom. Considere que o campo da imagem tenha as dimensões dadas pela figura (./img/q3_anexo.png)
## Gere manualmente correspondências entre a imagem e um mapa 2d com dimensões dadas pela figura. Calcule a homografia resultante e aplique na imagem original. Exiba o resultado. Dica: leia este tutorial (https://medium.com/acmvit/how-to-project-an-image-in-perspective-view-of-a-background-image-opencv-python-d101bdf966bc)

In [2]:
img = cv.imread("./img/soccer.jpg")
field = cv.imread("./img/q3_anexo.png")

h_field, w_field = field.shape[:2]
resized_img = cv.resize(img, (w_field, h_field))

positions = []
count = 0
pts1 = []
pts2 = []
current_image = None

In [3]:
def draw_circle(event, x, y, flags, param):
    global positions, count, current_image

    if event == cv.EVENT_LBUTTONUP:
        if count >= 4:
            return
        cv.circle(current_image, (x,y), 2, (255, 0, 0), -1)

        positions.append((x,y))
        
        count += 1
        if count > 1:
            cv.line(current_image, positions[-2], positions[-1], 
                   (255, 0, 255), 2)

In [4]:
current_image = field.copy()
positions = []
count = 0
cv.namedWindow('image')
cv.setMouseCallback('image', draw_circle)

while(True):
    cv.imshow('image', current_image)
    k = cv.waitKey(20) & 0xFF
    if k == 27:
        break
cv.destroyAllWindows()

pts1 = np.float32(positions)
print("Collected points:\n", pts1)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/heron/Documentos/visao_computacional/env/lib/python3.12/site-packages/cv2/qt/plugins"


Collected points:
 [[ 50.  15.]
 [460.  16.]
 [459. 280.]
 [ 50. 282.]]


In [5]:
current_image = resized_img.copy()
positions = []
count = 0
cv.namedWindow('image')
cv.setMouseCallback('image', draw_circle)

while(True):
    cv.imshow('image', current_image)
    k = cv.waitKey(20) & 0xFF
    if k == 27:
        break
cv.destroyAllWindows()

pts2 = np.float32(positions)
print("Collected points:\n", pts2)

Collected points:
 [[298.  89.]
 [469. 216.]
 [183. 277.]
 [ 66. 119.]]


In [6]:
h, _ = cv.findHomography(pts2, pts1, cv.RANSAC, 5.0)
print(f"Matriz de homografia: \n", h)

Matriz de homografia: 
 [[ 5.68994173e-01  4.65171062e+00 -5.21694093e+02]
 [-1.36255694e+00  1.89386898e+00  2.56048152e+02]
 [-2.13210225e-04  3.38095452e-03  1.00000000e+00]]


In [7]:
mask_field = np.zeros((h_field, w_field), dtype=np.uint8)
cv.fillConvexPoly(mask_field, pts1.astype(int), 255)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(309, 516), dtype=uint8)

In [8]:
mask_img = np.zeros((h_field, w_field), dtype=np.uint8)
cv.fillConvexPoly(mask_img, pts2.astype(int), 255)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(309, 516), dtype=uint8)

In [9]:
warped_img = cv.warpPerspective(resized_img, h, (w_field, h_field))


In [10]:
field_bg = cv.bitwise_and(field, field, mask=cv.bitwise_not(mask_field))
img_fg = cv.bitwise_and(warped_img, warped_img, mask=mask_field)

In [11]:
result = cv.add(field_bg, img_fg)

# Mostra o resultado
while(True):
    cv.imshow('image', result)
    k = cv.waitKey(20) & 0xFF
    if k == 27:
        break
cv.destroyAllWindows()

## 4. Leia o seguinte tutorial de calibração de câmera: https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html
## Você vai precisar de um tabuleiro de xadrez (pode imprimir numa folha A4, e colar num papelão ou emplastificar para a geometria ficar fixa). Meça as dimensões do seu tabuleiro para calibrar a câmera, considerando que o tabuleiro sempre está no plano z = 0, e que o canto inferior esquerdo do tabuleiro é a origem (0, 0, 0). Após calibrar a câmera, vamos incluir um objeto virtual na imagem. Considere a seguinte equação paramétrica do círculo centrado no ponto (1.5W, 1.5H, 0), com raio r = 0.5W e contido no plano z = 0, onde H e W s ̃ao a as medidas da altura e largura do tabuleiro:

## p(θ) = (r cos θ + 1.5W, r sen θ + 1.5H, 0).

## Se assegure de que o círculo apareça na imagem, de acordo com a posição do xadrez na imagem. Para desenhar o círculo, varie o valor do angulo θ entre 0 e 2π para amostrar alguns pontos, e projete-os na imagem. Repita isso 3 vezes, variando o ângulo entre o vetor normal do tabuleiro e o eixo principal da câmera.